# Infection detection in abdominal surgery images using fuzzy sets

In this booklet, we set up all the experimentation for infection detection. The method, as well as a detailed explanation of the method, is taken from:

- M. González-Hidalgo, M. Munar, P. Bibiloni, G. Moyà-Alcover, A. Craus-Miguel and J. J. Segura-Sampedro, "Detection of infected wounds in abdominal surgery images using fuzzy logic and fuzzy sets," 2019 International Conference on Wireless and Mobile Computing, Networking and Communications (WiMob), Barcelona, Spain, 2019, pp. 99-106, doi: 10.1109/WiMOB.2019.8923289.
- M. González-Hidalgo, G. Moyà-Alcover, M. Munar, P. Bibiloni, A. Craus-Miguel, X. González-Argenté and J.J Segura-Sampedro, "Detection and Automatic Deletion of Staples in Images of Wound of Abdominal Surgery for m-Health Applications". In: Tavares, J., Natal Jorge, R. (eds) VipIMAGE 2019. VipIMAGE 2019. Lecture Notes in Computational Vision and Biomechanics, vol 34. Springer, Cham. https://doi.org/10.1007/978-3-030-32040-9_23

Note that these two methods receive an image containing only the wound. Since different neural networks have been trained, and it has been concluded that the Double UNet architecture has the best performance in wound detection, we will use this method as a wound mask generator.

## Libraries

In [ ]:
import cv2
import glob
import json
import numpy
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" # Use only if there are available GPUs, and select which one has to be used.
import skimage

from tqdm import tqdm
from typing import Dict, Tuple

#Custom library with the implementation of the considered CNN architectures.
#Important: It must be used the version 1.0 of this library.
from cnn_architectures.architectures.base.CNNModel import CNNModel
from cnn_architectures.architectures.models.double_unet.double_unet import DoubleUNet
from cnn_architectures.architectures.models.unet.unet import UNet

#Custom library with the implementation of the staples detection methods.
from staples_detection.base.staple_detection_methods import StapleDetectionMethod
from staples_detection.staple_detector import StapleDetector

#Custom library with the implementation of some well-known inpainting methods in the literature.
from inPYinting.base.inpainting_algorithms import InpaintingAlgorithm
from inPYinting.inpainter import Inpainter

#Custom library with the implementation of the fuzzy sets-based colour segmentation algorithms.
from colour_segmentation.base.segmentation_algorithm import SegmentationAlgorithm
from colour_segmentation.segmentator import Segmentator

## Dataset load

First, we establish the main path of the dataset. Naturally, this will depend on the location of the dataset within the user's computer.

In [ ]:
REDSCAR_DATASET_PATH = r"/home/marc/UIB_EXPERIMENTS/REDSCAR/SUBSETS/MACHINE_LEARNING_DATASET"

Now, we set up the routes for the Redscar Machine Learning train and test sets. This should not be modified, as it is intrinsic to the dataset itself. All image names are stored in a list. There is one list for train and one for test.

In [ ]:
TRAIN_DATASET = os.path.join(REDSCAR_DATASET_PATH, "train")
TEST_DATASET = os.path.join(REDSCAR_DATASET_PATH, "test")

In [ ]:
with open(os.path.join(REDSCAR_DATASET_PATH, "test.txt")) as test_images_names_file:
    test_images_names = [line.rstrip() for line in test_images_names_file]
    
with open(os.path.join(REDSCAR_DATASET_PATH, "train.txt")) as train_images_names_file:
    train_images_names = [line.rstrip() for line in train_images_names_file]

## Neural network-based models for staple and wound segmentation

We proceed to load the two models trained for staples and wound segmentation in images. The models to be considered are the following:
- For staples segmentation, the model based on the UNet architecture trained with 100 epochs.
- For wound segmentation, the model based on the Double UNet architecture trained with 100 epochs.

In both cases, the models considered are the ones that have offered the best performance in their respective tasks. All CNN models used in the study are implemented in the `cnn_architectures` library, so we must continue to use this library for model loading.

### UNet model for staples segmentation

In [ ]:
unet_model_path = "/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/UNET/EXPERIMENT4/unet3_epochs=100_lr=3e-4_res=0.h5"
unet_model = UNet(input_size=(512, 512, 3),
                  out_channel=1, 
                  batch_normalization=True)
unet_model.build(n_filters=64, dilation_rate=1, layer_depth = 5, last_activation="sigmoid")
unet_model.load_weight(unet_model_path)

### Double UNet model for wound segmentation

In [ ]:
doubleunet_model_path = "/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1/dun3_epochs=100_lr=3e-5_res=0.h5"

doubleunet_model = DoubleUNet(input_size=(256, 256, 3))
doubleunet_model.build()
doubleunet_model.load_weight(doubleunet_model_path)

Although the `CNNModel` object of the `cnn_architectures` library already has a predefined method for prediction, it predicts and returns the mask in the size of the network. To avoid doing size transformations on the binarised image, we propose the following method to predict and return the result in the same size. The `predict_binary` method of `CNNModel` has been slightly adapted.

In [ ]:
def generate_prediction(image: numpy.ndarray, model: CNNModel, binary_threshold: float=0.5) -> Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray]:
    # The input image is resized to fit the size of the network.
    resized_image_normalized = skimage.transform.resize(image, (model.input_size[0], model.input_size[1], model.input_size[2]))
    
    # The prediction is made.
    prediction = model.model.predict(numpy.array([resized_image_normalized]))
    if isinstance(model, DoubleUNet):
        prediction = prediction[0][:, :, 1] # Remember: The Double UNet presents two outputs. It is the second that is intended to be obtained.
    else:
        prediction = prediction[0][:, :, 0]
        
    # It is resized again to generate a mask the same as the input image.
    final_prediction_raw = skimage.transform.resize(prediction, (image.shape[0], image.shape[1]))
    final_prediction_binary = final_prediction_raw >= binary_threshold
    final_prediction_int = (final_prediction_binary*255).astype(numpy.uint8)
    
    return final_prediction_raw, final_prediction_binary, final_prediction_int

## Experimentation

For each image we have, we will follow the method proposed in both reference works. Specifically, the steps to follow are:

1. Predict the location of the staples and wound using the pre-trained UNet and Double UNet models, respectively. 
2. The wound located in step (1) may have different connected components. Then, for each connected component:
    1. Obtain the coordinates of the bounding box that delimits it. This is only to be able to obtain the region of the wound to be studied, and will therefore be the input for the previous methods.
    2. Remove the staples using the mask obtained in the first step. 
    3. With the staples removed, Liu and Shamir's chromatic segmentation methods, based on fuzzy set membership functions, are applied. 

In [ ]:
from matplotlib import pyplot as plt

def process_image(image: numpy.ndarray, image_name: str, cnn_model_staples: UNet, cnn_model_wound: DoubleUNet) -> Dict:
    results = {}
    results["IMAGE_NAME"] = image_name
    
    # Computation of the mask with the position of the wound
    _, _, prediction_mask_staples = generate_prediction(image=image, model=cnn_model_staples)
    _, _, prediction_mask_wound = generate_prediction(image=image, model=cnn_model_wound)
    
    # Computation of the connected components of the mask
    #number_connected_components, labels_connected_components = cv2.connectedComponents(prediction_mask_wound)
    connected_component_output = cv2.connectedComponentsWithStats(prediction_mask_wound)
    number_connected_components = connected_component_output[0]
    labels_connected_components = connected_component_output[1]
    areas_connected_components = connected_component_output[2][:,cv2.CC_STAT_AREA]
    
    # Uncomment the following line to add the predicted mask to the dictionary of results
    # results["PREDICTED_WOUND_MASK": prediction_mask_wound]
    results["NUMBER_CONNECTED_COMPONENTS"] = number_connected_components-1
    
    # According to the documentation of connectedComponents provided by OpenCV, the 0 index is always reserved to background
    for connected_component_label in range(1, number_connected_components):
        
        # We select the corresponding component mask, and calculate the staple mask as the 
        # intersection of the component wound mask and the global staple mask.
        wound_segment_mask = numpy.zeros((image.shape[0], image.shape[1]), dtype=numpy.uint8)
        wound_segment_mask[labels_connected_components == connected_component_label] = 255
        
        staples_mask = prediction_mask_staples & wound_segment_mask
        
        # The coordinates of this connected component are obtained
        contours = cv2.findContours(wound_segment_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0][0]
        
        x,y,w,h = cv2.boundingRect(contours)
        wound_region = image[y:y+h, x:x+w, :]
        staples_mask = staples_mask[y:y+h, x:x+w]
        
        
        inpainter = Inpainter(image=wound_region, mask=staples_mask.astype(numpy.uint8))
        inpainting_result = inpainter.inpaint(InpaintingAlgorithm.NAVIER_STOKES)
        
        segmentator = Segmentator(image=inpainting_result.inpainted_image)
        result_liu = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_LIU,
                                         remove_achromatic_colours=False)
        result_shamir = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_SHAMIR,
                                            remove_achromatic_colours=False)
        result_chamorro = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_CHAMORRO,
                                              remove_achromatic_colours=False)
        result_amante = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_AMANTE,
                                            remove_achromatic_colours=False)
        
        result_liu_chr = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_LIU,
                                             remove_achromatic_colours=True)
        result_shamir_chr = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_SHAMIR,
                                                remove_achromatic_colours=True)
        result_chamorro_chr = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_CHAMORRO,
                                                  remove_achromatic_colours=True)
        result_amante_chr = segmentator.segment(method=SegmentationAlgorithm.FUZZY_SET_AMANTE,
                                                remove_achromatic_colours=True)
        
        # Uncomment the following lines to add the partial results to the dictionary of results
        # results[rf"WOUND_region={connected_component_label}"] = wound_region
        # results[rf"STAPLES_region={connected_component_label}"] = staples_mask
        # results[rf"INPAINTING_region={connected_component_label}"] = inpainting_result.inpainted_image
        # results[rf"LIU_COLOR_SEGMENTATION_region={connected_component_label}"] = result_liu.segmented_image
        # results[rf"SHAMIR_COLOR_SEGMENTATION_region={connected_component_label}"] = result_shamir.segmented_image
        
        results[rf"LIU_COLOR_REDPROP_region={connected_component_label}"] = result_liu.get_colour_proportion(colour_label=0)
        results[rf"SHAMIR_COLOR_REDPROP_region={connected_component_label}"] = result_shamir.get_colour_proportion(colour_label=0)
        results[rf"CHAMORRO_COLOR_REDPROP_region={connected_component_label}"] = result_chamorro.get_colour_proportion(colour_label=0)
        results[rf"AMANTE_COLOR_REDPROP_region={connected_component_label}"] = result_amante.get_colour_proportion(colour_label=0)
        
        results[rf"LIU_COLOR_CHR_REDPROP_region={connected_component_label}"] = result_liu_chr.get_colour_proportion(colour_label=0)
        results[rf"SHAMIR_COLOR_CHR_REDPROP_region={connected_component_label}"] = result_shamir_chr.get_colour_proportion(colour_label=0)
        results[rf"CHAMORRO_COLOR_CHR_REDPROP_region={connected_component_label}"] = result_chamorro_chr.get_colour_proportion(colour_label=0)
        results[rf"AMANTE_COLOR_CHR_REDPROP_region={connected_component_label}"] = result_amante_chr.get_colour_proportion(colour_label=0)
        
    return results

Finally, we run all the images. Attention: this cell has a high computational cost.

In [ ]:
experiment_train_images_results = []

for image_name in tqdm(train_images_names):
    image_path = os.path.join(TRAIN_DATASET, "IMAGES", image_name)
    image = cv2.imread(image_path)
    
    result = process_image(image=image, 
                           image_name=image_name, 
                           cnn_model_staples=unet_model, 
                           cnn_model_wound=doubleunet_model)
    experiment_train_images_results.append(result)

with open('train_redness_evaluation.json', 'w') as fout:
    json.dump(experiment_train_images_results, fout)

In [ ]:
experiment_test_images_results = []

for image_name in tqdm(test_images_names):
    image_path = os.path.join(TEST_DATASET, "IMAGES", image_name)
    image = cv2.imread(image_path)
    
    result = process_image(image=image, 
                           image_name=image_name, 
                           cnn_model_staples=unet_model, 
                           cnn_model_wound=doubleunet_model)
    experiment_test_images_results.append(result)

with open('test_redness_evaluation.json', 'w') as fout:
    json.dump(experiment_test_images_results, fout)